In [1]:
from masterfile.archive import MasterFile

new, ref = MasterFile.update()

Query Confirmed Planet Table...Done
Query extended database....Done
Building new reference table...Done
Completing database with extended database (may take some time)...

Done


In [3]:
new.by_pl_name('WASP-14 b', name_key='pl_name')['pl_tranmid','pl_tranmiderr1', 'pl_tranmiderr2']

pl_tranmid,pl_tranmiderr1,pl_tranmiderr2
d,d,d
float64,float64,float64
2456034.2126,0.00015,-0.00015


In [2]:
ref.by_pl_name('WASP-14 b', name_key='pl_name')['pl_tranmid','pl_tranmiderr1', 'pl_tranmiderr2']

pl_tranmid,pl_tranmiderr1,pl_tranmiderr2
bytes42,bytes42,bytes42
Wong et al. 2015,Wong et al. 2015,Wong et al. 2015
